In [2]:
import pandas as pd
import numpy as np
from pathlib import Path 
from tqdm import tqdm_notebook
from fastai import *
from fastai.text import *

In [3]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x

def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

mispell_dict = {"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"}

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = _get_mispell(mispell_dict)
def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

In [4]:
lm = True

# Language Model

In [6]:
if lm: path = Path('../input'); 
list(path.iterdir())

[WindowsPath('../input/embeddings'),
 WindowsPath('../input/glove.840B.300d'),
 WindowsPath('../input/sample_submission.csv'),
 WindowsPath('../input/test.csv'),
 WindowsPath('../input/train.csv')]

In [7]:
train_df = pd.read_csv(path/'train.csv')
test_df = pd.read_csv(path/'test.csv')
allText_df = train_df.copy().append(test_df.copy())

allText_df["question_text"] = allText_df["question_text"].apply(lambda x: x.lower())
allText_df["question_text"] = allText_df["question_text"].apply(lambda x: clean_text(x))
allText_df["question_text"] = allText_df["question_text"].apply(lambda x: clean_numbers(x))
allText_df["question_text"] = allText_df["question_text"].apply(lambda x: replace_typical_misspell(x))

#train_df["question_text"] = train_df["question_text"].apply(lambda x: x.lower())
#train_df["question_text"] = train_df["question_text"].apply(lambda x: clean_text(x))
#train_df["question_text"] = train_df["question_text"].apply(lambda x: clean_numbers(x))
#train_df["question_text"] = train_df["question_text"].apply(lambda x: replace_typical_misspell(x))

C:\Users\xw5735\AppData\Roaming\Python\Python36\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [8]:
if lm:
    bs = 48
    data_lm = (TextList.from_df(allText_df, path, cols='question_text')
                .split_by_rand_pct(0.1)
                .label_for_lm()           
                .databunch(bs=bs))

In [9]:
if lm:
    data_lm.save('tmp_lm')
    #data_lm = load_data('.','tmp_lm', bs=bs)

In [12]:
data_lm = load_data(path,'tmp_lm', bs=bs)

In [13]:
data_lm.show_batch()

idx,text
0,"mountain bike by just changing the tyres ? xxbos is gaza slowly becoming auschwitz , dachau or xxunk for palestinians ? xxbos why does quora automatically ban conservative opinions when reported , but does not do the same for liberal views ? xxbos is it crazy if i wash or wipe my groceries off ? germs are everywhere . xxbos is there such a thing as dressing moderately , and"
1,are the chances of a multiverse ? xxbos why do doctors hate earphones ? xxbos what would you do if you realize that you were worse at something than you thought ? xxbos why xxunk were not found in city ? xxbos what should one expect from neet if they ’ ve not got good rank in all india mock test ? xxbos earth ’ s magnetic field has been
2,"i do when i don ’ t know what i am passionate about due to depression ? xxbos what , as a white person with liberal leanings , makes you feel you deserve to live in xxrep 4 # ? it seems you have no purpose other than to apologize for existing . xxbos how can i help a friend who is suffering from a nervous breakdown ? xxbos how"
3,m great and when i feel bad my whole world is hell . what is this called and how do i go about seeing things more realistically and stopping the extremes ? xxbos are there good job oppurtunities for chip designing ? xxbos what job can an uneducated person do at a hospital ? xxbos what insect bite causes pus ? xxbos how can i seduce a dog ? xxbos
4,? xxbos how can i get the neighbor ' s dog to stop barking ? xxbos i am currently a gis analyst and my employer wants me to get the xxunk certification . is the xxunk certification worth it ? how important is the xxunk ? xxbos why does my dog keeps licking her breasts ? xxbos what does “ # # - second spot ” mean in the advertising


In [112]:
embed_size = 300
max_features = 60000
embedding_path = "../input/embeddings/glove.840B.300d.txt"

In [113]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embedding_index = dict(get_coefs(*o.split(" ")) for o in open(embedding_path, encoding='utf-8', errors='ignore'))
# all_embs = np.stack(embedding_index.values())
# emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std = -0.005838499, 0.48782197
word_index = data_lm.vocab.stoi
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words + 1, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [ ]:
if lm:
    learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult=0.3)
    learn.unfreeze()

In [ ]:
if lm: learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7))

In [ ]:
if lm: learn.save('language_model')

In [ ]:
if lm: learn.save_encoder('lm_encoder')

# Classifier

In [ ]:
class TextClasDataBunch(TextDataBunch):
    "Create a `TextDataBunch` suitable for training an RNN classifier."
    @classmethod
    def create(cls, train_ds, valid_ds, test_ds=None, path:PathOrStr='.', bs=64,val_bs=None, pad_idx=1, pad_first=True,
               device=None,no_check:bool=False, shuffle=[True, True, False],backwards=False, **kwargs) -> DataBunch:
        "Function that transform the `datasets` in a `DataBunch` for classification."
        datasets = [train_ds, valid_ds, test_ds]
        collate_fn = partial(pad_collate, pad_idx=pad_idx, pad_first=pad_first)
        train_sampler = SortishSampler(datasets[0].x, key=lambda t: len(datasets[0][t][0].data), bs=bs//2)
        train_dl = DataLoader(datasets[0], batch_size=bs//2, sampler=train_sampler, drop_last=True, **kwargs)
        dataloaders = [train_dl]
        dataloaders.append(DataLoader(datasets[1], batch_size=bs, **kwargs))
        dataloaders.append(DataLoader(datasets[2], batch_size=bs, **kwargs))
        return cls(*dataloaders, path=path, collate_fn=collate_fn)
    
TextList._bunch = TextClasDataBunch

In [ ]:
path = Path('../input'); list(path.iterdir())

In [ ]:
if lm:
    train_df = pd.read_csv(path/'train.csv').sample(frac=0.3, random_state=42)
else:
    train_df = pd.read_csv(path/'train.csv').sample(frac=0.9, random_state=42)

train_df["question_text"] = train_df["question_text"].apply(lambda x: x.lower())
train_df["question_text"] = train_df["question_text"].apply(lambda x: clean_text(x))
train_df["question_text"] = train_df["question_text"].apply(lambda x: clean_numbers(x))
train_df["question_text"] = train_df["question_text"].apply(lambda x: replace_typical_misspell(x))
#train0 = train_df[train_df.target==0].sample(n=100000, random_state=42)
#train1 = train_df[train_df.target==1].sample(n=100000, random_state=42, replace=True)
#train = pd.concat((train0, train1)); len(train)
#train.reset_index(inplace=True, drop=True)

In [ ]:
test_df = pd.read_csv(path/'test.csv')
test_df["question_text"] = test_df["question_text"].apply(lambda x: x.lower())
test_df["question_text"] = test_df["question_text"].apply(lambda x: clean_text(x))
test_df["question_text"] = test_df["question_text"].apply(lambda x: clean_numbers(x))
test_df["question_text"] = test_df["question_text"].apply(lambda x: replace_typical_misspell(x))

In [ ]:
test = TextList.from_df(test_df, path, cols='question_text')

In [ ]:
if lm:
    data = (TextList.from_df(train_df, path, cols='question_text', vocab=data_lm.vocab)
                    .split_by_rand_pct(0.1)
                    .label_from_df(cols=2)
                    .add_test(test)
                    .databunch(path='.')) 
else:
    data = (TextList.from_df(train_df, path, cols='question_text')
                .split_by_rand_pct(0.1)
                .label_from_df(cols=2)
                .add_test(test)
                .databunch(path='.')) 

In [ ]:
f_score = FBeta(beta=1)

In [ ]:
learn = text_classifier_learner(data,arch=AWD_LSTM, drop_mult=0.5, metrics=[accuracy, f_score])
if lm: learn.load_encoder('lm_encoder')
learn.freeze()

In [ ]:
gc.collect();

In [ ]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))

In [ ]:
preds = learn.get_preds(DatasetType.Valid)
proba = to_np(preds[0][:,1])
ytrue = to_np(preds[1])

In [ ]:
import sklearn
from sklearn.metrics import *
from sklearn.metrics import roc_curve, precision_recall_curve
def threshold_search(y_true, y_proba, plot=False):
    precision, recall, thresholds = precision_recall_curve(y_true, y_proba)
    thresholds = np.append(thresholds, 1.001) 
    F = 2 / (1/precision + 1/recall)
    best_score = np.max(F)
    best_th = thresholds[np.argmax(F)]
    if plot:
        plt.plot(thresholds, F, '-b')
        plt.plot([best_th], [best_score], '*r')
        plt.show()
    search_result = {'threshold': best_th , 'f1': best_score}
    return search_result 

In [ ]:
thr = threshold_search(ytrue, proba, plot=True); thr

In [ ]:
preds = learn.get_preds(DatasetType.Test)
proba = to_np(preds[0][:,1])
predsC = (proba > thr['threshold']).astype(int)

In [ ]:
sub = pd.read_csv('../input/sample_submission.csv')
sub.prediction = predsC
sub.to_csv("submission.csv", index=False)

In [ ]:
sub.head()